In [1]:
import os
import requests
import csv
import re
import pandas as pd

In [2]:
station_table = pd.read_parquet('../data/station_table.parquet')
stations = station_table['station_id']
data_url = 'https://www.ndbc.noaa.gov/data/realtime2/'

In [35]:
def download_buoy_data(buoy_id):
    buoy_data_url = os.path.join(data_url, buoy_id + ".txt")
    response = requests.get(buoy_data_url)
    if response.status_code == 200:
        with open("../data/buoys/txt/" + buoy_id + ".txt", "wb") as file:
            file.write(response.content)
        print(f"Downloaded data for buoy {buoy_id}")
        return True
    else:
        print(f"Failed to download data for buoy {buoy_id}")
        return False

In [ ]:
active_stations = []
for station in stations:
    if download_buoy_data(station.upper()) == True:
        active_stations.append(station)
    

In [6]:
len(active_stations)

927

In [38]:
def process_parc(station_id):
    df = pd.read_csv("../data/buoys/txt/" + station_id.upper() + ".txt", sep=r'\s+')
    df.to_parquet("../data/buoys/parquet/" + station_id.upper() + ".parquet")
    return df

In [40]:
buoy_dict = {}
for station in active_stations:
    buoy_dict[station] = process_parc(station)
    

In [73]:
def convert_columns_to_float(df):
    for column in df.columns:
        try:
            df[column] = df[column].astype(float)
        except ValueError:
            df[column] = pd.to_numeric(df[column], errors='coerce')
    return df


In [75]:
for buoy_id, buoy_df in buoy_dict.items():
    buoy_dict[buoy_id] = convert_columns_to_float(buoy_df)